In [42]:
def create_permutation_matrix(s):
    s = list(s)
    def key(si):
        return si[0]
    s_p = [(si,i) for i,si in enumerate(s)]
    s_p.sort(key = key)
    l = len(s)
    M = MatrixSpace(ZZ,l)
    P = M([[0]*l]*l)
    for i in range(l):
        P[i,s_p[i][1]] = 1
    return P

In [43]:
def Basis(F0,s = None):
    m,n = F0.dimensions()
    if (s == None):
        s = [0]*m

    pi = create_permutation_matrix(s)
    P,L,U = (pi*F0).LU()
    r = U.rank()
    Lr = L[:r,:r]
    G = L[r:,:r]
    K = F0.base_ring()
    pR.<x> = K[]
    Pinv = P.inverse_of_unit()
    pi_inv = pi.inverse_of_unit()
    M = pi_inv*P*block_matrix(2,2,[[x,0],[-G*Lr.inverse(), 1]])*Pinv*pi
    MS = MatrixSpace(ZZ,m)
    u = pi_inv*P*(Pinv*pi*vector(s) + vector([1]*r+[0]*(m-r)))
    return M,u

In [44]:
m = 8
n = 5
F = GF(13)
pR.<x> = F[]

In [45]:
M = MatrixSpace(F,m,n)
MX = MatrixSpace(pR,m,n)

In [46]:
AX=MX.random_element()
A = M(AX.coefficient_matrix(0))

In [47]:
A = M([[10, 1 , 6,  0, 12],
[ 3 , 6 , 4 , 6 ,10],
[ 9 , 6 ,11,  4,  2],
[ 5  ,4  ,6 , 4 , 0],
[ 9 , 9,  4  ,7 , 9],
[11  ,6 ,10 , 0,  1],
[ 8  ,8 , 1 , 1,  2],
[ 9 ,10 , 3, 12,  8]]
)

In [48]:
shifts = [-i for i in range(m)]
shifts

[0, -1, -2, -3, -4, -5, -6, -7]

In [49]:
B, new_shifts = Basis(A,shifts)

In [50]:
B.is_minimal_approximant_basis(A,1, shifts = shifts)

True

In [51]:
new_shifts

(0, 12, 11, 11, 10, 9, 8, 7)

In [55]:
for i in range(100):
    AX=MX.random_element()
    A = M(AX.coefficient_matrix(0))
    shifts = [ZZ.random_element(-4,5) for i in range(m)]
    B, new_shifts = Basis(A,shifts)
    if(B.row_degrees(shifts = shifts) != new_shifts.list()):
        print('False',B.row_degrees(shifts = shifts),new_shifts.list())
    if (not B.is_reduced(shifts = shifts)):
        print('Not reduced')
        break

False [-1, -2, 4, 3, 3, 3, 3, -3] [12, 9, 4, 3, 3, 3, 3, 10]
Not reduced


In [56]:
pR.<x> = GF(13)[]
m = 8
n = 5
sigma = 40
Ms = MatrixSpace(pR,m,n)
A = Ms.random_element()

In [57]:
def M_Basis(F,sigma,shift = None):
    pR = F.base_ring()
    x = pR.gen()
    m,n = F.dimensions()
    R = parent(F)
    if (shift == None):
        shift = [0]*m
        
    P = [0]*sigma
    u = [0]*sigma
    M = [0]*sigma
    P[0],u[0] = Basis(F.coefficient_matrix(0), shift)
    for k in range(1,sigma):
        F_prime = R(x^(-k)*P[k-1]*F).coefficient_matrix(0)
        M[k],u[k] = Basis(F_prime, u[k-1])
        P[k] = M[k]*P[k-1]
    return P, u, M

In [58]:
k = 0
for i in range(100):
    sigma = ZZ.random_element(1,50)
    shifts = [ZZ.random_element(-4,5) for i in range(m)]
    A = Ms.random_element()
    P,u, M = M_Basis(A,sigma, shift = shifts)
    if (not P[-1].is_minimal_approximant_basis(A,sigma, shifts = shifts)):
        print("Error")
        k += 1
k

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


95

In [59]:
P[-1].is_reduced(shifts = shifts)

False

In [67]:
M[1].row_degrees(shifts = u[0])

[12, 12, 12, 12, 12, 12, 12, 12]

In [69]:
u[0]

(2, 2, 11, 4, 0, 0, 3, 0)

In [62]:
M[1].degree_matrix()

[ 1 -1 -1 -1 -1 -1 -1 -1]
[-1  1 -1 -1 -1 -1 -1 -1]
[ 0  0  0 -1  0  0 -1  0]
[ 0  0 -1  0  0  0 -1  0]
[-1 -1 -1 -1  1 -1 -1 -1]
[-1 -1 -1 -1 -1  1 -1 -1]
[ 0  0 -1 -1  0  0  0  0]
[-1 -1 -1 -1 -1 -1 -1  1]

In [65]:
M[1]

[ x  0  0  0  0  0  0  0]
[ 0  x  0  0  0  0  0  0]
[ 6  4  1  0 12  4  0  2]
[12  4  0  1  4 11  0  1]
[ 0  0  0  0  x  0  0  0]
[ 0  0  0  0  0  x  0  0]
[ 9  2  0  0  5 12  1  2]
[ 0  0  0  0  0  0  0  x]

In [76]:
M[1].row_degrees(u[0].list())

[12, 12, 12, 12, 12, 12, 12, 12]

In [81]:
L = M[1].shift(u[0].list(), row_wise = False)

In [82]:
L.row_degrees()

[3, 3, 11, 4, 1, 1, 3, 1]

In [83]:
M

[
   [ x  0  0  0  0  0  0  0]  [x 0 0 0 0 0 0 0]
   [ 0  x  0  0  0  0  0  0]  [0 x 0 0 0 0 0 0]
   [ 6  4  1  0 12  4  0  2]  [1 6 1 0 9 3 0 7]
   [12  4  0  1  4 11  0  1]  [4 8 0 1 0 8 0 8]
   [ 0  0  0  0  x  0  0  0]  [0 0 0 0 x 0 0 0]
   [ 0  0  0  0  0  x  0  0]  [0 0 0 0 0 x 0 0]
   [ 9  2  0  0  5 12  1  2]  [1 4 0 0 8 0 1 3]
0, [ 0  0  0  0  0  0  0  x], [0 0 0 0 0 0 0 x],

[ x  0  0  0  0  0  0  0]  [ 1 12  0  3  4  4  0  4]
[12  1  0  0  9  0  9  7]  [ 0  x  0  0  0  0  0  0]
[ 9  0  1  0  7  2  8  7]  [ 0  3  1 12  3  5  0  9]
[ 4  0  0  1  3  1  4  3]  [ 0  0  0  x  0  0  0  0]
[ 0  0  0  0  x  0  0  0]  [ 0  0  0  0  x  0  0  0]
[ 0  0  0  0  0  x  0  0]  [ 0  0  0  0  0  x  0  0]
[ 0  0  0  0  0  0  x  0]  [ 0  6  0  1  3  7  1 12]
[ 0  0  0  0  0  0  0  x], [ 0  0  0  0  0  0  0  x],

[ x  0  0  0  0  0  0  0]  [ 1  7  0  6 12 11 10  0]
[ 6  1  0  0  9  7 10  8]  [ 0  x  0  0  0  0  0  0]
[ 1  0  1  0  5 12  3 12]  [ 0  1  1  3  8 10  4  0]
[ 7  0  0  1  7  2 11  5]  

In [85]:
P[-1].leading_positions(shifts = shifts)

[0, 1, 6, 3, 4, 5, 6, 7]